# Example OpenFAST Post-Processing Python Notebook

In [1]:
import numpy as np
import pandas as pd
import os, glob, shutil
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.autolayout'] = True

In [2]:
## PSD function
def psdsignal(time,sensor):
    #Frequency vector
    dfFFT=1/(np.max(time)-np.min(time))
    fFFTlength = len(time)
    fFFT = np.linspace(0,dfFFT*(fFFTlength-1),fFFTlength)
    # Power spectral density
    psd = 2*(1/fFFTlength)*np.abs(np.fft.fft(sensor))**2
    return psd,fFFT

In [ ]:
# Compare Plot results

outname = 'MHK_RM1_Floating.out'
paths = ['Your_Run_Path']
lines = ['-','--',':','-.']

outcols = [['Wind1VelX','Wind1VelY','Wind1VelZ'],['BldPitch1','BldPitch2'],['RtSpeed'],['PtfmSurge','PtfmSway','PtfmHeave','PtfmRoll','PtfmPitch','PtfmYaw'],['OoPDefl1','TwrTpTDxi'],['RtSkew'],
           ['GenPwr','GenTq'],['RtFldFxh','RtFldFyh','RtFldFzh']]
outunits = [['m/s','m/s','m/s'],['deg','deg'],['rpm'],['m','m','m','deg','deg','deg'],['m','m'],['deg'],
            ['kW','kN-m'],['N','N','N']]

fig,axs = plt.subplots(len(outcols),2,figsize=(20,len(outcols)*2),gridspec_kw={'width_ratios':[5, 2]})    
for i in range(len(paths)):
    OF = pd.read_csv(paths[i]+outname,skiprows=[0,1,2,3,4,5,7],sep='\t')
    with open(paths[i]+outname,"r") as fst:
        ttotal = float(fst.readlines()[5].split(' ')[0])
    if OF['Time'][len(OF['Time'])-1] < ttotal*0.99: print('Run '+str(i)+' Failed')
    else: print('Run '+str(i)+' Finished')
    for col in range(len(outcols)):
        for c in range(len(outcols[col])):
            axs[col,0].plot(OF.Time,OF[outcols[col][c]],label=outcols[col][c]+' '+outunits[col][c],ls=lines[i])
            [psd,fFFT] = psdsignal(OF.Time,OF[outcols[col][c]]); 
            axs[col,1].plot(fFFT[5:-5],psd[5:-5],label=outcols[col][c]+' '+outunits[col][c],ls=lines[i])
            axs[col,1].set(ylabel='PSD',xlim=[0,1])
        axs[col,0].set_prop_cycle(None)
        axs[col,1].set_prop_cycle(None)
        if i == 0: axs[col,1].legend(loc=2,framealpha=0.7)
axs[col,0].set(xlabel='Time [s]')
plt.show()